<h2 style=' color: crimson;font-family: Colonna MT; font-weight: 600; font-size: 35px; text-align: Center'>Invoice Generator</h2>

---


<h2 style='font-family: Colonna MT; font-weight: 600; font-size: 25px; text-align: left'>1.0. Import Required Libraries and Modules</h2>

In [16]:
from typing import List, Dict, Optional, Union
from Invoice.Invoicepdf import PDFInvoice
from datetime import datetime
import os

- Now, let's generate some demo invoices to test and demonstrate how our invoice module works. We'll use the `PDFInvoice` class from the `Invoice.Invoicepdf` module to create a professional-looking PDF invoice. The process begins by defining sample invoice items, including services like website design, SEO services, content writing, and more, each with their respective unit prices and quantities. These items are structured as a list of dictionaries, making it easy to modify or expand the invoice contents as needed. 

- Next, we initialize the `PDFInvoice` object and configure its margins to ensure proper spacing. A new page is added to the PDF, and the invoice header is populated with the client's name and business details using the `add_invoice_info` method. The invoice items are then formatted into a clean, readable table using `add_table`, which also calculates the subtotal automatically. Finally, the invoice summary section is added, which includes the subtotal, tax (set at 10% in this example), and a discount of $50.00. The completed invoice is saved to the "Outputs" directory as "Invoice.pdf," providing a tangible output that verifies our code's functionality. This demo effectively showcases how the module handles data input, calculations, and PDF generation, making it ready for integration into larger systems or further customization.

<h2 style='font-family: Colonna MT; font-weight: 600; font-size: 25px; text-align: left'>1.0. Generate Invoice</h2>

In [17]:
from Invoice.Invoicepdf import PDFInvoice

# Sample invoice items
invoice_items = [
    {"description": "Website Design", "unit_price": 500, "quantity": 1},
    {"description": "SEO Services", "unit_price": 300, "quantity": 2},
    {"description": "Content Writing", "unit_price": 150, "quantity": 3},
    {"description": "Logo Design", "unit_price": 250, "quantity": 1},
    {"description": "Hosting (1 Year)", "unit_price": 120, "quantity": 1},
    {"description": "Social Media Management", "unit_price": 200, "quantity": 2},
    {"description": "Email Marketing Campaign", "unit_price": 180, "quantity": 1},
    {"description": "Maintenance & Support", "unit_price": 100, "quantity": 4},
]


# Generate PDF
pdf = PDFInvoice()
pdf.set_margins(left=15, top=15, right=15)
pdf.add_page()
pdf.add_invoice_info(to_name="Client Name", from_name="Your Business")
subtotal = pdf.add_table(invoice_items)
pdf.add_summary_section(subtotal=subtotal, tax_rate=0.10, discount=50.00)
pdf.output("Outputs/Invoice.pdf")

<h2 style='font-family: Colonna MT; font-weight: 600; font-size: 25px; text-align: left'>1.0. Error Handling Protocols</h2>

**Now, let’s implement error handling protocols before generating invoices.**

Before we even begin creating an invoice, it’s important to make sure that all the input data is valid, complete, and logical. This helps us avoid unexpected issues later in the process and ensures the generated invoice is accurate and professional.

To do this, we’ll set up a series of checks and validations that act as our first line of defense. These will include:

1. **Required Field Checks** – We’ll verify that all critical fields like item descriptions, prices, and quantities are provided and not left empty.
2. **Data Type and Value Checks** – We’ll ensure that numeric values such as prices, quantities, tax rates, and discounts are valid (e.g., positive numbers, not strings or negative values).
3. **Structural Integrity** – We’ll confirm that the invoice structure includes customer or business information, as generating an invoice without this would be meaningless.
4. **Graceful Feedback** – If any data is missing or invalid, we won’t proceed with invoice creation. Instead, we’ll raise a clear, custom error that explains exactly what went wrong. This helps both users and developers quickly identify and fix the issue.

By enforcing these protocols, we can avoid silent failures or confusing results. It ensures we only proceed when we’re confident that the input data meets all necessary requirements. This leads to cleaner, safer, and more reliable invoice generation.

In [3]:
class InvoiceGenerationError(Exception):
    pass
def validate_invoice_item(item: Dict) -> None:
    required_fields = ['description', 'unit_price', 'quantity']
    for field in required_fields:
        if field not in item:
            raise ValueError(f"Missing required field in invoice item: {field}")
    
    if not isinstance(item['description'], str) or not item['description'].strip():
        raise ValueError("Description must be a non-empty string")
    
    if not isinstance(item['unit_price'], (int, float)) or item['unit_price'] <= 0:
        raise ValueError("Unit price must be a positive number")
    
    if not isinstance(item['quantity'], int) or item['quantity'] <= 0:
        raise ValueError("Quantity must be a positive integer")

def validate_invoice_data(customer: str, business: str, invoice_items: List[Dict], 
                          tax_rate: float, discount: Union[float, int]) -> None:
    if not isinstance(customer, str) or not customer.strip():
        raise ValueError("Customer name must be a non-empty string")
    if not isinstance(business, str) or not business.strip():
        raise ValueError("Business name must be a non-empty string")
    if not isinstance(invoice_items, list) or not invoice_items:
        raise ValueError("Invoice items must be a non-empty list")
    for item in invoice_items: validate_invoice_item(item)
    if not isinstance(tax_rate, (int, float)) or tax_rate < 0:
        raise ValueError("Tax rate must be a non-negative number")
    if not isinstance(discount, (int, float)) or discount < 0:
        raise ValueError("Discount must be a non-negative number")

def generate_invoice(customer: str, business: str,invoice_items: List[Dict],
    tax_rate: float = 0.10, discount: float = 50.0) -> Optional[PDFInvoice]:
    try:
        validate_invoice_data(customer, business, invoice_items, tax_rate, discount)
        pdf = PDFInvoice()
        pdf.set_margins(left=15, top=15, right=15)
        pdf.add_page()
        pdf.add_invoice_info(customer, business)
        subtotal = pdf.add_table(invoice_items)
        if subtotal is None or subtotal <= 0: raise InvoiceGenerationError("Invalid subtotal calculated from invoice items")
        pdf.add_summary_section(subtotal=subtotal, tax_rate=tax_rate, discount=discount)
        return pdf 
    except Exception as e:
        error_msg = f"Failed to generate invoice: {str(e)}"
        raise InvoiceGenerationError(error_msg) from e

# Sample invoice items
invoice_items = [
    {"description": "Website Design", "unit_price": 500, "quantity": 1},
    {"description": "SEO Services", "unit_price": 300, "quantity": 2},
    {"description": "Content Writing", "unit_price": 150, "quantity": 3},
    {"description": "Logo Design", "unit_price": 250, "quantity": 1},
    {"description": "Hosting (1 Year)", "unit_price": 120, "quantity": 1},
    {"description": "Social Media Management", "unit_price": 200, "quantity": 2},
    {"description": "Email Marketing Campaign", "unit_price": 180, "quantity": 1},
    {"description": "Maintenance & Support", "unit_price": 100, "quantity": 4},
]


def main():
    customer, business = 'Maximilian Ngoko', "Saggittarius"
    tax_rate, discount = 0.10, 50.00
    
    os.makedirs("Outputs/Invoices", exist_ok=True)
    try:
        invoice = generate_invoice(
            customer=customer,
            business=business,
            invoice_items=invoice_items,
            tax_rate=tax_rate,
            discount=discount
        )
        
        if invoice:
            timestamp = datetime.now().strftime("%Y%m%d")
            filename = f"INV2006{timestamp}.pdf"
            filepath = os.path.join("Outputs/Invoices", filename)
            invoice.output(filepath)
            print(f"Invoice generated successfully! Saved to: {filepath}")
        return invoice
    
    except InvoiceGenerationError as e:
        print(f"Error generating invoice: {e}")
        return None

if __name__ == "__main__":
    main()

Invoice generated successfully! Saved to: Outputs/Invoices\INV200620250509.pdf


<h2 style='font-family: Colonna MT; font-weight: 600; font-size: 25px; text-align: left'>1.0. Generating Multiple Invoices</h2>

Sometimes, there’s a need to generate multiple invoices at once—such as when billing several clients in bulk or processing a batch of transactions. To handle this scenario efficiently, we can extend our invoice generation system to support batch processing. This means instead of manually running the invoice logic for each client or transaction, we’ll design our program to loop through a list of invoice data entries, validating and generating each one automatically.

During this process, we’ll apply the same strict error handling protocols to each invoice individually. That way, if one invoice has issues (like missing data or invalid values), it won’t stop the entire batch. Instead, the system will log the specific error for that invoice, skip it, and continue processing the rest. This makes the system much more robust and scalable. At the end of the run, we can provide a summary—indicating how many invoices were successfully created and flagging any that failed along with the reasons why. This approach is not only more efficient but also improves user experience by automating what would otherwise be a repetitive, error-prone task.

Would you like help drafting the Python code for this multi-invoice generation flow?


In [15]:
from typing import List, Dict, Optional, Tuple
from datetime import datetime
import os
from pathlib import Path
import logging

class ClientInvoiceGenerator:
    """A class to handle client invoice generation with robust error handling and logging."""
    def __init__(
        self,
        business_name: str,
        invoice_items: List[Dict[str, Union[str, float, int]]],
        output_dir: str = "invoices",
        tax_rate: float = 0.10,
        discount: float = 50.00,
        invoice_prefix: str = "INV200"
    ):
        """Initialize the invoice generator with configuration."""
        self.business_name = self._validate_business_name(business_name)
        self.invoice_items = self._validate_invoice_items(invoice_items)
        self.output_dir = output_dir
        self.tax_rate = tax_rate
        self.discount = discount
        self.invoice_prefix = invoice_prefix
        self._setup_logging()
        self._ensure_output_directory()
        
    def _setup_logging(self) -> None:
        """Configure basic logging for invoice generation."""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)
        
    def _ensure_output_directory(self) -> None:
        """Ensure the output directory exists."""
        try:
            Path(self.output_dir).mkdir(parents=True, exist_ok=True)
        except OSError as e:
            self.logger.error(f"Failed to create output directory: {str(e)}")
            raise
            
    @staticmethod
    def _validate_business_name(name: str) -> str:
        """Validate the business name."""
        if not name.strip():
            raise ValueError("Business name cannot be empty")
        return name.strip()
        
    @staticmethod
    def _validate_invoice_items(items: List[Dict]) -> List[Dict]:
        """Validate invoice items structure."""
        if not items:
            raise ValueError("Invoice items list cannot be empty")
            
        required_keys = {'description', 'unit_price', 'quantity'}
        for item in items:
            if not all(key in item for key in required_keys):
                raise ValueError(f"Each invoice item must contain {required_keys}")
                
            if not isinstance(item['description'], str):
                raise ValueError("Description must be a string")
                
            if item['unit_price'] < 0 or item['quantity'] < 0:
                raise ValueError("Prices and quantities must be positive")
                
        return items
        
    def _generate_invoice_filename(self) -> Tuple[str, str]:
        """Generate a unique invoice filename and path."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{self.invoice_prefix}_{timestamp}.pdf"
        filepath = os.path.join(self.output_dir, filename)
        return filename, filepath
        
    def generate_for_client(self, client: str) -> Optional[str]:
        """Generate an invoice for a single client."""
        try:
            if not client.strip():
                raise ValueError("Client name cannot be empty")
                
            filename, filepath = self._generate_invoice_filename()
            
            invoice = generate_invoice(
                customer=client.strip(),
                business=self.business_name,
                invoice_items=self.invoice_items,
                tax_rate=self.tax_rate,
                discount=self.discount
            )
            
            invoice.output(filepath)
            self.logger.info(f"Successfully generated invoice for {client}: {filepath}")
            return filepath
            
        except Exception as e:
            self.logger.error(f"Failed to generate invoice for {client}: {str(e)}")
            return None
            
    def generate_for_clients(self, clients: List[str]) -> Dict[str, Optional[str]]:
        """Generate invoices for multiple clients with comprehensive reporting."""
        if not clients:
            raise ValueError("Client list cannot be empty")
            
        results = {}
        success_count = 0
        
        for client in clients:
            filepath = self.generate_for_client(client)
            results[client] = filepath
            if filepath:
                success_count += 1
                
        self.logger.info(
            f"Invoice generation complete. Success: {success_count}/{len(clients)}"
        )
        return results

if __name__ == "__main__":
    try:
        # Configuration
        CLIENTS = ["Client A", "Client B", "Client C"]
        BUSINESS = "Saggittarius"
        OUTPUT_DIR = "Outputs/Invoices"
        
        INVOICE_ITEMS = [
            {"description": "Website Design", "unit_price": 500, "quantity": 1},
            {"description": "SEO Services", "unit_price": 300, "quantity": 2},
            {"description": "Content Writing", "unit_price": 150, "quantity": 3}
        ]
        
        # Initialize and run generator
        generator = ClientInvoiceGenerator(
            business_name=BUSINESS,
            invoice_items=INVOICE_ITEMS,
            output_dir=OUTPUT_DIR,
            tax_rate=0.10,
            discount=50.00
        )
        
        results = generator.generate_for_clients(CLIENTS)
        
        # Print summary
        print("\nInvoice Generation Summary:")
        for client, path in results.items():
            status = "✓ SUCCESS" if path else "✗ FAILED"
            print(f"{status} - {client}: {path or 'Not generated'}")
            
    except Exception as e:
        logging.error(f"Critical error in invoice generation: {e}")
        raise

2025-05-09 12:41:19,994 - INFO - Successfully generated invoice for Client A: Outputs/Invoices\INV200_20250509_124119.pdf
2025-05-09 12:41:20,153 - INFO - Successfully generated invoice for Client B: Outputs/Invoices\INV200_20250509_124119.pdf
2025-05-09 12:41:20,183 - INFO - Successfully generated invoice for Client C: Outputs/Invoices\INV200_20250509_124120.pdf
2025-05-09 12:41:20,186 - INFO - Invoice generation complete. Success: 3/3



Invoice Generation Summary:
✓ SUCCESS - Client A: Outputs/Invoices\INV200_20250509_124119.pdf
✓ SUCCESS - Client B: Outputs/Invoices\INV200_20250509_124119.pdf
✓ SUCCESS - Client C: Outputs/Invoices\INV200_20250509_124120.pdf


In [13]:
from Invoice.Invoicepdf import PDFInvoice
from typing import List, Dict, Optional
from datetime import datetime
from pathlib import Path
import pandas as pd
import os

class InvoiceGenerator:
    """A class to handle invoice generation from customer data."""
    def __init__(
        self,
        data_path: str = "Datasets/customer_data.csv",
        output_dir: str = "Outputs/Invoices",
        business_name: str = "Your Business",
        tax_rate: float = 0.10,
        discount: float = 0.00,
        sample_size: Optional[int] = 50
    ):
        """Initialize the invoice generator with configuration."""
        self.data_path = data_path
        self.output_dir = output_dir
        self.business_name = business_name
        self.tax_rate = tax_rate
        self.discount = discount
        self.sample_size = sample_size
        self._ensure_output_directory()
        
    def _ensure_output_directory(self) -> None:
        """Ensure the output directory exists."""
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)
        
    def _prepare_invoice_items(self, group: pd.DataFrame) -> List[Dict[str, Union[str, float]]]:
        """Prepare invoice items from a DataFrame group."""
        return [
            {
                'description': row['Product'],
                'unit_price': row['UnitPrice'],
                'quantity': row['Quantity']
            }
            for _, row in group.iterrows()
        ]
        
    def _generate_pdf_invoice(
        self,
        customer: str,
        cart_id: str,
        invoice_items: List[Dict[str, Union[str, float]]]
    ) -> str:
        """Generate a PDF invoice and return the filename."""
        pdf = PDFInvoice()
        pdf.set_margins(left=15, top=15, right=15)
        pdf.add_page()
        
        # Add invoice information
        pdf.add_invoice_info(
            to_name=customer,
            from_name=self.business_name
        )
        
        # Add items and calculate subtotal
        subtotal = pdf.add_table(invoice_items)
        
        # Add summary section
        pdf.add_summary_section(
            subtotal=subtotal,
            tax_rate=self.tax_rate,
            discount=self.discount
        )
        
        # Generate filename and save
        filename = f"INV00-{cart_id}.pdf"
        filepath = os.path.join(self.output_dir, filename)
        pdf.output(filepath)
        
        return filename
        
    def _log_success(self, filename: str) -> None:
        """Log successful invoice generation."""
        total_length = 100
        save_message = "Invoice Generated Successfully  ✔✔✔✔"
        remaining_length = total_length - len(filename) - len(save_message) - 3
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"{current_time}: {filename} : {'-' * remaining_length} {save_message}")
        
    def generate_invoices(self) -> None:
        """Main method to generate all invoices from the data source."""
        try:
            # Load and prepare data
            customer_df = pd.read_csv(self.data_path)
            df = customer_df.head(self.sample_size).copy() if self.sample_size else customer_df
            
            # Group by Customer and CartID
            grouped = df.groupby(['Customer', 'CartID'])
            
            # Process each group
            success_count = 0
            for (customer, cart_id), group in grouped:
                try:
                    invoice_items = self._prepare_invoice_items(group)
                    filename = self._generate_pdf_invoice(customer, cart_id, invoice_items)
                    self._log_success(filename)
                    success_count += 1
                except Exception as e:
                    print(f"Error generating invoice for {customer} (Cart {cart_id}): {str(e)}")
                    continue
                    
            print(f"\nInvoice generation complete. Successfully generated {success_count}/{len(grouped)} invoices.")
            
        except Exception as e:
            print(f"Fatal error in invoice processing: {str(e)}")
            raise

# Example usage
if __name__ == "__main__":
    generator = InvoiceGenerator(
        business_name="Mizer Companition",
        tax_rate=0.10,
        discount=50.00
    )
    generator.generate_invoices()

2025-05-09 12:34:22: INV00-CART-1469.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:22: INV00-CART-3986.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:22: INV00-CART-3331.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:23: INV00-CART-2871.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:23: INV00-CART-2357.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:23: INV00-CART-6849.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:23: INV00-CART-9087.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34:23: INV00-CART-9133.pdf : ------------------------------------------ Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:34

In [11]:
import os
import pandas as pd
from datetime import datetime

def generate_and_save_invoices(input_filepath="Datasets/customer_data.csv", sample_size=50):
    try:
        customer_df = pd.read_csv(input_filepath)
        df = customer_df.head(sample_size).copy()
        grouped = df.groupby(['Customer', 'CartID'])
        successful_invoices = 0
        for (customer, cart_id), group in grouped:
            try:
                invoice_items = [
                    {
                        'description': row['Product'],
                        'unit_price': row['UnitPrice'],
                        'quantity': row['Quantity']
                    }
                    for _, row in group.iterrows()
                ]
                
                if process_single_invoice(customer, cart_id, invoice_items):
                    successful_invoices += 1
                    
            except Exception as e:
                log_error(f"Error processing invoice for {customer} (CartID: {cart_id}): {str(e)}")
                continue
                
        print(f"\nInvoice generation complete. Success: {successful_invoices}/{len(grouped)}")
        
    except Exception as e:
        log_error(f"Fatal error in invoice processing: {str(e)}")
        return False

    return True

def process_single_invoice(customer, cart_id, invoice_items):
    BUSINESS_NAME = "Mizer Caompanition"
    TAX_RATE = 0.10
    DISCOUNT = 50.00
    output_dir = "Outputs/Invoices"
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        invoice = generate_invoice(
            customer=customer,
            business=BUSINESS_NAME,
            invoice_items=invoice_items,
            tax_rate=TAX_RATE,
            discount=DISCOUNT
        )
        
        if not invoice:
            raise InvoiceGenerationError("Empty invoice returned")
        
        timestamp = datetime.now().strftime("%Y%m%d")
        filename = f"INV2006 - {cart_id}.pdf"
        filepath = os.path.join(output_dir, filename)
        invoice.output(filepath)
        log_success(filename)
        return True
        
    except InvoiceGenerationError as e:
        log_error(f"Invoice generation failed for {customer} (CartID: {cart_id}): {str(e)}")
        return False

def log_success(filename):
    total_length = 100
    text_length = len(filename)
    save_message = "Invoice Generated Successfully  ✔✔✔✔"
    remaining_length = total_length - text_length - len(save_message) - 3
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"{current_time}: {filename} : {'-' * remaining_length} {save_message}")

def log_error(message):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"{current_time}: ERROR: {message}")


if __name__ == "__main__":
    generate_and_save_invoices()

2025-05-09 12:30:16: INV2006 - CART-1469.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-3986.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-3331.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-2871.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-2357.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-6849.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-9087.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30:16: INV2006 - CART-9133.pdf : -------------------------------------- Invoice Generated Successfully  ✔✔✔✔
2025-05-09 12:30